In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.2 MB/s 


In [ ]:
import os
path="/content/drive/MyDrive/PVT-tensorflow2-main"
os.chdir(path)
os.listdir(path)


['.gitignore',
 'README.md',
 'model',
 '.idea',
 'generator',
 'saved_models',
 'dataset',
 'checkpoints',
 'utils',
 'requirements.txt',
 '.ipynb_checkpoints',
 '1.png',
 '2.png',
 'transformer.h5',
 'train.py',
 '10605.png',
 '20605.png',
 'pvt0605.h5']

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.4.0

Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.2+zzzcolab20220527125636.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.7 MB 16 kB/s 
     |████████████████████████████████| 14.8 MB 37.0 MB/s 
     |████████████████████████████████| 462 kB 41.2 MB/s 
     |████████████████████████████████| 132 kB 63.1 MB/s 
     |███████

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.8.2


In [ ]:
# %matplotlib inline
# %matplotlib notebook

# !python train.py

In [ ]:
def get_generator():
    train_generator = CIFARGenerator( mode="train")
    val_generator = CIFARGenerator( mode="valid")
    return train_generator, val_generator

In [ ]:

import tensorflow as tf
import tensorflow_addons as tfa

def get_optimizers():
    warmup_epochs =0
    warmup_lr = 1e-5
    init_lr = 1e-4
    weight_decay=0
    optimizer = tfa.optimizers.AdamW(learning_rate=init_lr,name='adam',weight_decay=weight_decay)

    return optimizer


In [ ]:
import tensorflow as tf
import numpy as np
def get_lr_scheduler(lr_scheduler):
    warmup_epochs =0
    warmup_lr = 1e-5
    init_lr = 1e-4
    if lr_scheduler == 'step':
        def scheduler(epoch,lr=0.001):
            if epoch < warmup_epochs:
                current_epoch_lr = warmup_lr + epoch * (init_lr -warmup_lr) / warmup_epochs
                print("epoch:{} lr:{:.6f}".format(epoch,current_epoch_lr.numpy()))
                return current_epoch_lr
            else:
                for index, val in enumerate(lr_decay_epoch):
                    if epoch < val:
                        current_epoch_lr = init_lr*lr_decay**index
                        print("epoch:{} lr:{:.6f}".format(epoch,current_epoch_lr.numpy()))
                        return current_epoch_lr
            current_epoch_lr = init_lr*lr_decay**len(lr_decay_epoch)
            print("epoch:{} lr:{:.6f}".format(epoch,current_epoch_lr.numpy()))
            return current_epoch_lr
        return scheduler
    elif lr_scheduler == 'cosine':
        def scheduler(epoch,lr=0.001):
            if epoch < warmup_epochs:
                current_epoch_lr = warmup_lr + epoch * (init_lr - warmup_lr) / warmup_epochs
            else:
                current_epoch_lr = init_lr * (
                        1.0 + tf.math.cos(np.pi / (epochs - warmup_epochs) * (epoch - warmup_epochs))) / 2.0
            print("epoch:{} lr:{:.6f}".format(epoch,current_epoch_lr.numpy()))
            return current_epoch_lr
    else:
        raise ValueError("{} is not supported!".format(lr_scheduler))
    return scheduler


In [ ]:
from model.efficientnet import EfficientNet
from model.resnet import ResNet
from model.PVT import PVTNet
from model.VIT import VITNet
from model.resnet_for_cifar import ResNetForCifar
def get_model(mode,num_class):

    model = PVTNet(img_size=32, classes=num_class,type=mode.split('-')[-1],pretrain=None)
    return model.get_model()
def get_model(mode,num_class):

    model = VITNet(img_size=32, classes=num_class,type=mode.split('-')[-1],pretrain=None)
    return model.get_model()


In [ ]:

def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images

def load_images():
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(
    #     buffer_size=10000).batch(batch_size)
    # test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

    return train_images, train_labels, test_images, test_labels

In [ ]:
import argparse
import tensorflow as tf
import logging
logging.getLogger().setLevel(logging.INFO)

import sys
import os
import numpy as np 
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sb
from keras.preprocessing.image import ImageDataGenerator

x_train, y_train, x_test, y_test = load_images()
datagen = ImageDataGenerator(
		featurewise_center=False,  # set input mean to 0 over the dataset
		samplewise_center=False,  # set each sample mean to 0
		featurewise_std_normalization=False,  # divide inputs by std of the dataset
		samplewise_std_normalization=False,  # divide each input by its std
		zca_whitening=False,  # apply ZCA whitening
		rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
		width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
		height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
		horizontal_flip=True,  # randomly flip images
		vertical_flip=False)  # randomly flip images
	# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)
model = VITNet(img_size=32, classes=10,pretrain=None)
model = model.get_model()
# model = get_model('PVT-tiny',10)
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = get_optimizers()
lr_scheduler = get_lr_scheduler('cosine')

    # lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, verbose=1, min_lr=0)
lr_cb = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

model.compile(optimizer,loss_object,metrics=["accuracy"])
history = model.fit(x_train,y_train,
                          validation_data=(x_test, y_test),
                          epochs=200,batch_size=128,
                          callbacks=[],
                          verbose=1,
                          )
plt.plot(history.history["accuracy"], label="training")
plt.plot(history.history["val_accuracy"], label="validation")
plt.title("Googlenet")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.savefig ('10605.png') 
plt.legend()
plt.show()
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="validation loss")
plt.title("Googlenet")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.savefig ('20605.png') 
plt.legend()
plt.show()
test_predict = model.predict(x_test)
y_test_labels = np.argmax(test_predict, axis=1)
y_test = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test, y_test_labels)
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print("Confusion Matrix\n", cm)
df_c_matrix = pd.DataFrame(cm, index=[clss for clss in class_labels], columns=[clss for clss in class_labels])
plt.figure(figsize=(10, 8))
sb.heatmap(df_c_matrix, annot=True)
plt.show()
model.save_weights('pvt0605.h5')
# model.load_weights('transformer.h5')


32 10


AttributeError: ignored

In [ ]:
train_generator, val_generator = get_generator()
for i in range(10):
  img = CIFARGenerator( mode="train")



In [ ]:
(_, train_labels), (_, test_labels) = tf.keras.datasets.cifar10.load_data()
train_generator, val_generator = get_generator()
train_images =  train_generator.trainimg()
test_images =  val_generator.trainimg()
xtrain_output = model.predict(train_images)
